In [2]:
import os

from datasets import load_dataset
from yt_dlp import YoutubeDL
from yt_dlp.utils import DownloadError

dataset = load_dataset("json", data_files="data/vivid_instruct_65k_raw.jsonl")["train"]

print(dataset)


def extract_duration(video_id):
  with YoutubeDL(params={"format": "best[ext=mp4]", "quiet": True}) as ydl:
    try:
      video_info = ydl.extract_info(
        f"https://www.youtube.com/watch?v={video_id}", download=False
      )
      video_duration = video_info.get("duration", -1)
      return video_duration
    except DownloadError:
      return -1


def map_fn(batch):
  temp = {}
  durations = []
  for video_id in batch["id"]:
    duration = extract_duration(video_id)
    durations.append(duration)

  temp["id"] = batch["id"]
  temp["category"] = batch["category"]
  temp["duration"] = durations
  return temp


# data = dataset.select(range(10))
data = dataset.map(
  map_fn,
  batched=True,
  batch_size=16,
  num_proc=os.cpu_count(),
  remove_columns=["id", "category"],
)

Parameter 'function'=<function map_fn at 0x7f7328291990> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Dataset({
    features: ['id', 'category'],
    num_rows: 7943
})


Map (num_proc=12):   0%|          | 0/7943 [00:00<?, ? examples/s]

ERROR: [youtube] iOsBf4dl1gs: Private video. Sign in if you've been granted access to this video
ERROR: [youtube] DX24Yqr_vKE: Private video. Sign in if you've been granted access to this video
ERROR: [youtube] mpQt9WCFlp0: Video unavailable. This video contains content from Netflix, who has blocked it in your country on copyright grounds
ERROR: [youtube] Hu1yPZqHR74: Video unavailable. This video has been removed by the uploader
ERROR: [youtube] WfTUPpXM5iM: Video unavailable. This video has been removed by the uploader
ERROR: [youtube] CfVOgHkpJb8: Private video. Sign in if you've been granted access to this video
ERROR: [youtube] TNbRqIpMGjA: Private video. Sign in if you've been granted access to this video
ERROR: [youtube] l156aB5QHLo: Private video. Sign in if you've been granted access to this video
ERROR: [youtube] yFY33Xyf0Ig: This live stream recording is not available.
ERROR: [youtube] nJrBvKjGqck: Private video. Sign in if you've been granted access to this video
ERROR: [yo

In [5]:
print(data.filter(lambda x: x.get("duration", -1) > -1))

Filter:   0%|          | 0/7943 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'category', 'duration'],
    num_rows: 7871
})


In [6]:
data.filter(lambda x: x.get("duration", -1) > -1).to_json(
  "data/vivid_instruct_65k_test.jsonl", index=False
)

Creating json from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

473371